# 股票预测模型工作流

---
### 工作流说明
1.  **阶段零 (Setup)**: 导入库、加载配置。
2.  **阶段一 (Data Pipeline)**: 独立运行。负责处理并保存数据，生成 L2 特征数据缓存。
3.  **阶段二 (Model Pipeline)**: 独立运行。包含三个子步骤：
    - **2.1 HPO**: 自动调参。
    - **2.2 (预处理)**: 智能地加载或生成 L3 预处理数据缓存
    - **2.3 (模型训练)**: 使用 L3 缓存进行高效的模型训练。
    - **2.4 (评估)**: 对训练结果进行聚合与可视化。

## 0. 通用设置与导入

In [ ]:
import sys
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

# 设置 Matplotlib 样式 (可以在 main_train 外部设置)
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# --- 动态导入 main_train ---
project_root = str(Path().resolve())
if project_root not in sys.path:
    sys.path.append(project_root)

# 从我们统一的核心引擎中导入所有需要的函数和模块
from main_train import (
    run_load_config_and_modules,
    run_all_data_pipeline,
    run_preprocess_l3_cache,
    run_hpo_train,
    run_all_models_train,
    run_performance_evaluation,
    run_results_visualization
)

# --- 加载全局配置和模块 ---
# 在 Notebook 的生命周期中，我们只加载一次
config, modules = run_load_config_and_modules()

if not (config and modules):
    raise RuntimeError("环境初始化失败，请检查配置文件路径和模块导入。")

# **阶段一：数据准备与特征工程**

In [2]:
USE_LATEST_DATE = True
if config and modules:
    print(f"--- INFO: 日期模式 -> {'使用最新日期' if USE_LATEST_DATE else '使用配置文件中的固定日期'} ---")
    
    # 将开关状态传递给核心引擎函数
    run_all_data_pipeline(config, modules, use_today_as_end_date=USE_LATEST_DATE)

  - INFO: 已将 sh.601360 (sh.601360) 的数据缓存至 data\data_cache\raw_ohlcv\raw_sh.601360_2010-10-26_2025-10-26.pkl
  - 正在从 Baostock 下载 sh.601377 (sh.601377) 的日线行情...
  - INFO: 已将 sh.601377 (sh.601377) 的数据缓存至 data\data_cache\raw_ohlcv\raw_sh.601377_2010-10-26_2025-10-26.pkl
  - 正在从 Baostock 下载 sh.601390 (sh.601390) 的日线行情...
  - INFO: 已将 sh.601390 (sh.601390) 的数据缓存至 data\data_cache\raw_ohlcv\raw_sh.601390_2010-10-26_2025-10-26.pkl
  - 正在从 Baostock 下载 sh.601398 (sh.601398) 的日线行情...
  - INFO: 已将 sh.601398 (sh.601398) 的数据缓存至 data\data_cache\raw_ohlcv\raw_sh.601398_2010-10-26_2025-10-26.pkl
  - 正在从 Baostock 下载 sh.601600 (sh.601600) 的日线行情...
  - INFO: 已将 sh.601600 (sh.601600) 的数据缓存至 data\data_cache\raw_ohlcv\raw_sh.601600_2010-10-26_2025-10-26.pkl
  - 正在从 Baostock 下载 sh.601601 (sh.601601) 的日线行情...
  - INFO: 已将 sh.601601 (sh.601601) 的数据缓存至 data\data_cache\raw_ohlcv\raw_sh.601601_2010-10-26_2025-10-26.pkl
  - 正在从 Baostock 下载 sh.601607 (sh.601607) 的日线行情...
  - INFO: 已将 sh.601607 (sh.601607) 的数据缓存至 data

d:\Project\Command\Python\Neural\Wolf_of_Wall_Street\data_process\get_data.py:135: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns_df = closes_df.pct_change().fillna(0)


--- 市场广度数据已生成并缓存至 data\data_cache\market_breadth_2010-10-26_2025-10-26.pkl ---
  - 正在为 SPY 获取美股数据...
  - 正在从 Yahoo Finance 下载 SPY 的数据...


d:\Project\Command\Python\Neural\Wolf_of_Wall_Street\data_process\get_data.py:331: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start_date, end=end_date_inclusive, progress=False)

1 Failed download:
['SPY']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


  - 警告 [YF]: 下载 SPY 数据失败 (尝试 1/3). 将在 0.50 秒后重试. 错误: yfinance returned an empty DataFrame for SPY.


d:\Project\Command\Python\Neural\Wolf_of_Wall_Street\data_process\get_data.py:331: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start_date, end=end_date_inclusive, progress=False)

1 Failed download:
['SPY']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


  - 警告 [YF]: 下载 SPY 数据失败 (尝试 2/3). 将在 1.00 秒后重试. 错误: yfinance returned an empty DataFrame for SPY.


d:\Project\Command\Python\Neural\Wolf_of_Wall_Street\data_process\get_data.py:331: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start_date, end=end_date_inclusive, progress=False)

1 Failed download:
['SPY']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


  - 警告 [YF]: 下载 SPY 数据失败 (尝试 3/3). 将在 2.00 秒后重试. 错误: yfinance returned an empty DataFrame for SPY.
  - 错误 [YF]: 下载 SPY 数据在 3 次尝试后仍失败。
--- 正在获取市场情绪数据 (恐慌指数) ---
  - 错误 [akshare]: 获取市场情绪数据失败: module 'akshare' has no attribute 'idx_ivix'
--- 所有全局市场数据准备完毕 ---


--- 正在为 TCL科技 (000100.SZ) 生成【训练用】历史特征 ---
  - 数据窗口: 2010-10-26 to 2025-10-26
  - 正在从本地缓存加载 TCL科技 (sz.000100) 的原始日线数据...
  - INFO: 正在为 'TCL科技的行业指数' (159928.SZ) 获取指数数据...
  - 正在从 Baostock 下载 TCL科技的行业指数 (159928.SZ) 的日线行情...
  - WARNING [BS]: 未能获取到 TCL科技的行业指数 (159928.SZ) 在指定日期范围的数据。
    - 警告: 无法直接获取指数 'TCL科技的行业指数' (159928.SZ)。将尝试在本地合成等权重指数...
    - 错误: 在 API 映射表中未找到指数 'TCL科技的行业指数' (159928.SZ) 的成分股查询接口。
  - INFO: 正在为 '基准指数' (sh.000300) 获取指数数据...
  - 正在从 Baostock 下载 基准指数 (sh.000300) 的日线行情...
  - INFO: 已将 基准指数 (sh.000300) 的数据缓存至 data\data_cache\raw_ohlcv\raw_sh.000300_2010-10-26_2025-10-26.pkl
    - SUCCESS: 已直接从 Baostock 获取到 '基准指数' (sh.000300) 的数据。
INFO: 开始特征计算流水线...
  - [Calculating Features] Running: Technical Indicators...
    - Calcul

# **阶段二：模型训练与评估**

### 2.1 数据预加载与全局预处理 (L3 缓存)

In [3]:
if config and modules:
    # 调用核心引擎中的预处理函数
    # force_reprocess=True/False 可以方便地控制是否重建缓存
    global_data_cache = run_preprocess_l3_cache(config, modules, force_reprocess=False)

=== 工作流阶段 2.1：为模型预处理数据 (L3 缓存) ===
INFO: 开始执行预处理流程 (分块保存模式)...



正在预处理股票:   0%|          | 0/8 [00:00<?, ?it/s]

--- INFO: L3 缓存文件已在磁盘上准备就绪。 ---
--- 阶段 2.1 成功完成。 ---


### 2.2 超参数优化

In [4]:
# 同样，可以通过一个简单的开关来控制是否运行
RUN_HPO = False

if RUN_HPO and config and modules and 'global_data_cache' in locals():
    # 调用核心引擎中的 HPO 函数
    # HPO 的结果会自动更新到内存中的 config 字典里
    run_hpo_train(config, modules, global_data_cache)

### 2.3 模型训练

In [ ]:
if config and modules and 'global_data_cache' in locals():
    # 调用核心引擎中的模型训练函数
    # 同样可以通过开关控制是否强制重训
    all_ic_history = run_all_models_train(
        config, 
        modules,  
        force_retrain_base=False, 
        force_retrain_fuser=False
    )

=== 工作流阶段 2.3：训练所有模型 ===


处理股票模型:   0%|          | 0/8 [00:00<?, ?it/s]


--- 2.3.1 为 TCL科技 (000100.SZ) 训练基础模型 ---
--- 开始为 TCL科技 (000100.SZ) 进行 LGBM 模型训练 ---
INFO: 未检测到任何历史记录，将从头开始全新训练。
INFO: 开始对 TCL科技 (000100.SZ) 进行跨 62 folds 的前向验证...


正在 TCL科技 上训练 LGBM:   0%|          | 0/62 [00:00<?, ?it/s]

### 2.4 结果聚合、评估与可视化

In [ ]:
if config and modules and 'all_ic_history' in locals() and all_ic_history:
    # 调用核心引擎中的评估和可视化函数
    evaluation_summary, backtest_summary, final_eval_df = run_performance_evaluation(config, modules, all_ic_history)
    
    # 只有在有结果时才进行可视化
    if evaluation_summary is not None or backtest_summary is not None:
        run_results_visualization(config, modules, evaluation_summary, backtest_summary, final_eval_df)
    else: print('无可视化结果')